In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def CameraCalibrate_Distortion(Image): 
    '''
    Doing camera calibraion without needing any inputs and doing image undistortion for
    the input image
    output: undistorted image
    '''
    
    #camera calibration:
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob('camera_cal/calibration*.jpg')

    # Step through the list and search for chessboard corners
    for image in images:
        img = cv2.imread(image)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
    
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    
    #distortion
    #Image=cv2.cvtColor(Image, cv2.COLOR_BGR2RGB)
    dst=cv2.undistort(Image,mtx,dist,None,mtx)
    
    return dst


def plotter(image_1,image_2,title_1,title_2,image_name):
    '''
    a helper function used to show the outputs through the pipline steps
    '''
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
    ax1.set_title(title_1, fontsize=20)
    ax1.imshow(image_1,cmap='gray')
    ax2.set_title(title_2, fontsize=20)
    ax2.imshow(image_2,cmap='gray')
    
    output_path='output_images/'
    plt.savefig('{}/{}'.format(output_path,image_name))
    

In [ ]:
#Doing calibration and distortion
image=cv2.imread('camera_cal/calibration1.jpg')
undistorted=CameraCalibrate_Distortion(image)
plotter(image,undistorted,'input image','Undistorted image','distortion')

img_d = cv2.imread('./test_images/test1.jpg')
img_d=cv2.cvtColor(img_d, cv2.COLOR_BGR2RGB)
img_und=CameraCalibrate_Distortion(img_d) 
plotter(img_d,img_und,'input image','Undistorted image','distortion2')


In [3]:
#Getting the Binary threshold image to detect lane lines
#Applying Sobel x,y->Sobel magnitude and direction ->HLS Color threshold->Combine Sobel and threshold.
'''
The output is Binary image
'''
def abs_sobel_thresh(img, orient):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    kernel_size=3
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0,kernel_size))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1,kernel_size))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    thresh_min=30
    thresh_max=100
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1
    # Return the result
    return binary_output

def mag_dir_thresh(img):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Take both Sobel x and y gradients
    kernel_size=15
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0,kernel_size)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1,kernel_size)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    mag_thresh_min=70
    mag_thresh_max=120
    binary_mag_output = np.zeros_like(gradmag)
    binary_mag_output[(gradmag >= mag_thresh_min) & (gradmag <= mag_thresh_max)] = 1
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_dir_output =  np.zeros_like(absgraddir)
    dir_thresh_min=0.7
    dir_thresh_max=1.3  #mentioned in combining thresholds lesson
    binary_dir_output[(absgraddir >= dir_thresh_min) & (absgraddir <= dir_thresh_max)] = 1
    return binary_mag_output,binary_dir_output

def color_threshold(img):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    S = hls[:,:,2]
    thresh_s = (190,255)
    binary_s = np.zeros_like(S)
    binary_s[(S > thresh_s[0]) & (S <= thresh_s[1])] = 1
    H = hls[:,:,0]
    thresh_h = (15, 100)
    binary_h = np.zeros_like(H)
    binary_h[(H > thresh_h[0]) & (H <= thresh_h[1])] = 1
    
    return binary_s,binary_h

def combined_threshold(sobelx,sobely,magnitude,direction,Schannel,Hchannel):
    Binary_image = np.zeros_like(direction)
    Binary_image[((sobelx == 1) & (sobely == 1)) | ((magnitude == 1) & (direction == 1)) |((Schannel == 1) & (Hchannel == 1))] = 1
    return Binary_image

    

In [ ]:
img = cv2.imread('./test_images/test1.jpg')
img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

img_undistorted=CameraCalibrate_Distortion(img)                            ##img

sobel_x=abs_sobel_thresh(img_undistorted,orient='x')
sobel_y=abs_sobel_thresh(img_undistorted,orient='y')

magnitude,direction=mag_dir_thresh(img_undistorted)

binary_s,binary_h=color_threshold(img_undistorted)

Binary=combined_threshold(sobel_x,sobel_y,magnitude,direction,binary_s,binary_h) ###important:combined

plotter(sobel_x,sobel_y,'sobel x','sobel y','sobel gradient')
plotter(magnitude,direction,'magnitude','gradient','sobel magnitude and gradient')
plotter(binary_s,binary_h,'binary s','binary h','color threshold')
plotter(img_undistorted,Binary,'image','Binary image','lane detection')

In [ ]:
#Doing Perspective Transform : choosing four points manually to choose source and destination points.
plt.figure(figsize = (25,50))
plt.imshow(img_undistorted)
points=[[588,470], [245,719], [1142, 719],[734,470]]
for point in points:
    plt.plot(point[0],point[1],'or')
points.append(points[0])
x,y=zip(*points)    #make 2 lists trick
plt.plot(x,y,'r',linewidth=3)
print('The image size is {},{}'.format(img.shape[0],img.shape[1]))

In [4]:
def warp(img):
    img_size=(img.shape[1],img.shape[0])
    #four source coordinates
    src = np.float32([[588,470], [245,719], [1142, 719],[734,470]])   
    #four distance coordinates
    dst = np.float32([[320,0], [320,720], [960, 720],[960,0]])
    #compute the perspective transform,M:
    M=cv2.getPerspectiveTransform(src,dst)
    #compute the inverse
    Minv=cv2.getPerspectiveTransform(dst,src)
    #create warped image uses linear interpolation
    warped=cv2.warpPerspective(img,M,img_size,flags=cv2.INTER_LINEAR)
    return warped,Minv
    

In [ ]:
warped_undistorted,Minv=warp(img_undistorted)
warped_Binary,Minv=warp(Binary) #important


plotter(img_undistorted,warped_undistorted,'undistorted image','perspective transform','perspective1')
plotter(Binary,warped_Binary,'','','perspective2')

In [5]:
#Finding Lane Lines using histogram and sliding windows

def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram and These 
    #will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero  = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    ###DRAWING WINDOWS###
    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low        = binary_warped.shape[0] - (window+1)*window_height
        win_y_high       = binary_warped.shape[0] - window*window_height
        win_xleft_low    = leftx_current - margin
        win_xleft_high   = leftx_current + margin
        win_xright_low   = rightx_current - margin
        win_xright_high  = rightx_current + margin
        
        # Draw the windows on the visualization image
        #cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        #(win_xleft_high,win_y_high),(0,255,0), 4) 
        #cv2.rectangle(out_img,(win_xright_low,win_y_low),
        #(win_xright_high,win_y_high),(0,255,0), 4) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
    # Extract left and right line pixel positions the position of ones.
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    ## Visualization ##
    # Colors in the left and right lane regions
    #out_img[lefty, leftx] = [255, 0, 0]
    #out_img[righty, rightx] = [0, 0, 255]
    # Plots the left and right polynomials on the lane lines
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    out_img_windows= out_img.astype(int)

    
    ###FINDING THE LINES#####
    ##Grab activated pixels##
    ###Set the area of search based on activated x-values###
    ### within the +/- margin of our polynomial function###
    left_lane_inds_lines = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds_lines = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    # Again, extract left and right line pixel positions
    leftx_line   = nonzerox[left_lane_inds_lines]
    lefty_line   = nonzeroy[left_lane_inds_lines] 
    rightx_line  = nonzerox[right_lane_inds_lines]
    righty_line  = nonzeroy[right_lane_inds_lines]
    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty_line,leftx_line, 2)
    right_fit = np.polyfit(righty_line,rightx_line, 2)
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img_line = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img_line = np.zeros_like(out_img_line)
    # Color in left and right line pixels
    out_img_line[lefty_line,leftx_line] = [255, 0, 0]
    out_img_line[righty_line,rightx_line] = [0, 0, 255]
    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))
    
    #Draw the lane onto the warped blank image
    #cv2.fillPoly(window_img_line, np.int_([left_line_pts]), (0,255, 0))
    #cv2.fillPoly(window_img_line, np.int_([right_line_pts]), (0,255, 0))
    #Plot the polynomial lines onto the image
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    result_lane= cv2.addWeighted(out_img_line, 1, window_img_line, 0.3, 0)
    result_lane=result_lane.astype(int)
    ## End visualization steps ##
    
    return left_fitx,right_fitx

In [ ]:
#The sliding windows
leftx,rightx=find_lane_pixels(warped_Binary)
#plt.imshow(windows)

In [ ]:
#The detecte lanes
leftx,rightx=find_lane_pixels(warped_Binary)
#plt.imshow(lanes)

In [6]:
#Finding the lane curvature and the car offset of the centre of the lane 
'''
Take as input the warped binary image and the x position of the ones(lanes) in the 
binary warped image
return the raduis of curvature for left and right lane,the car offset.
'''

def lane_curvature(img,rightx,leftx):
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0])
    leftx = leftx[::-1]  # Reverse to match top-to-bottom in y
    rightx = rightx[::-1]  # Reverse to match top-to-bottom in y
    # Fit a second order polynomial to pixel positions in each fake lane line
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix,leftx*xm_per_pix,2)
    right_fit_cr =np.polyfit(ploty*ym_per_pix,rightx*xm_per_pix,2)
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    #the car offset 
    #the camera is at the centre of the lane so the car offset should be the difference between the lanes difference and 
    #the mid of the camera:
    diff1=(leftx[-1]+rightx[-1])/2
    diff2=img.shape[1]//2
    offset=((diff1-diff2)*xm_per_pix)
    return left_curverad,right_curverad,offset
    

In [14]:
#putting the information at each frame and unwarp it again with the detected lane.
def Drawing(original,warped,rightx,leftx,Minv,left_curverad,right_curverad,offset):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    # Recast the x and y points into usable format for cv2.fillPoly()
    ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0])
    pts_left = np.array([np.transpose(np.vstack([leftx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([rightx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (original.shape[1], original.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(original, 1, newwarp, 0.3, 0)
    cv2.putText(result, 'Left lane curvature: {:.4f}m'.format(left_curverad),(710,60),cv2.FONT_HERSHEY_DUPLEX,1,(255, 255,0),4)
    cv2.putText(result, 'right lane curvature: {:.4f}m'.format(right_curverad),(710,120),cv2.FONT_HERSHEY_DUPLEX,1,(255,255,0),4)
    cv2.putText(result, 'offset of car is: {:.4f}m'.format(offset),(710,180),cv2.FONT_HERSHEY_DUPLEX,1,(255,255,0),4)
    
    return result

In [ ]:
left_curverad,right_curverad,offset=lane_curvature(warped_Binary,rightx,leftx)
frame=Drawing(img,warped_Binary,rightx,leftx,Minv,left_curverad,right_curverad,offset)
plotter(img_undistorted,frame,'The original image','final frame','output')

In [15]:
#Make pipeline:Take as input an image and return the detected lane and the radius of curvature and car offset
def pip(picture):
    #1 Calibration and undistortion
    undistorted_img=CameraCalibrate_Distortion(picture)
    
    #2 Binary image
    sobel_x=abs_sobel_thresh(undistorted_img,orient='x')
    sobel_y=abs_sobel_thresh(undistorted_img,orient='y')
    magnitude,direction=mag_dir_thresh(undistorted_img)
    binary_s,binary_h=color_threshold(undistorted_img)
    
    Binary=combined_threshold(sobel_x,sobel_y,magnitude,direction,binary_s,binary_h)
    
    #3 perspective transform
    warped_Binary,Minv=warp(Binary)
    
    #4 Detect lanes
    leftx,rightx=find_lane_pixels(warped_Binary)
    
    #5
    left_curverad,right_curverad,offset=lane_curvature(warped_Binary,rightx,leftx)
    Final_frame=Drawing(undistorted_img,warped_Binary,rightx,leftx,Minv,left_curverad,right_curverad,offset)
    
    return Final_frame

In [ ]:
#Final test before video streaming:
#img = cv2.imread('./test_images/test5.jpg')
#img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#frame=pip(img)
#plotter(img,frame,'original','output frame','output2')

In [16]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

def process_image(picture):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    result=pip(picture)
    # you should return the final output (image where lines are drawn on lanes)
    return result

In [17]:
white_output = 'output_lane.mp4'

clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)


t:   0%|                                                                            | 0/1260 [00:00<?, ?it/s, now=None]

Moviepy - Building video output_lane.mp4.
Moviepy - Writing video output_lane.mp4



Moviepy - Done !
Moviepy - video ready output_lane.mp4
Wall time: 1h 26min 14s


In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))